### Notation:
- SAL- small area
- PP- police precinct
- AEA- Albers Equal Area Conic

In [1]:
from random import shuffle, randint
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from mpl_toolkits.basemap import Basemap
from shapely.geometry import Polygon, Point, MultiPoint, MultiPolygon, LineString, mapping, shape
from descartes import PolygonPatch
import random
import fiona
import numpy as np
import csv
from fiona import collection

import geopandas as gpd
from geopandas.tools import sjoin # rtree index in-build, used with inner, intersection
import pandas as pd

from collections import defaultdict


def sjoin(left_df, right_df, how='inner', op='intersects',
          lsuffix='left', rsuffix='right', **kwargs):
    """Spatial join of two GeoDataFrames.
    left_df, right_df are GeoDataFrames
    how: type of join
        left -> use keys from left_df; retain only left_df geometry column
        right -> use keys from right_df; retain only right_df geometry column
        inner -> use intersection of keys from both dfs;
                 retain only left_df geometry column
    op: binary predicate {'intersects', 'contains', 'within'}
        see http://toblerity.org/shapely/manual.html#binary-predicates
    lsuffix: suffix to apply to overlapping column names (left GeoDataFrame)
    rsuffix: suffix to apply to overlapping column names (right GeoDataFrame)
    """


In [2]:
def find_intersections(o):
    
    from collections import defaultdict

    paired_ind = [o.pp_index, o.sal_index]

    d_over_ind = defaultdict(list)

    # creating a dictionary that has prescints as keys and associated small areas as values
    for i in range(len(paired_ind[0].values)):
        if not paired_ind[0].values[i]==paired_ind[1].values[i]: # it shows itself as intersection
            d_over_ind[paired_ind[0].values[i]].append(paired_ind[1].values[i])

    # get rid of the pol precincts with no small areas associated to them- not the most efficient way
    d_temp = {}
    for l in d_over_ind:
        if len(d_over_ind[l]):
            d_temp[l] = d_over_ind[l]

    return d_temp
    
    
def calculate_join_indices(g1_reind, g2_reind):

        # A: region of the police data with criminal record
        # C: small area with population data
        # we look for all small areas intersecting a given C_i, calculate the fraction of inclusion, scale the
        # population accordingly: area(A_j, where A_j crosses C_i)/area(A_j)* popul(A_j)
        
    
        # the actual indexing:
        out = sjoin(g1_reind, g2_reind, how ="inner", op = "intersects")
        
        out.drop('index_right', axis=1, inplace=True) # there is a double index fo smal areas, so we drop one
        #out_sorted = out.sort(columns='polPrecincts_index', ascending=True) # guess sorting is not necessary, cause we are
        # using doctionaries at later stages
        #dict_over_ind = find_intersections(out_sorted)

        # output retains only 1 area (left or right join), and gives no intersection area.
        # so we create an array with paired indices: police precincts with associated small areas
        # we use it in a loop in a function below
        dict_over_ind = find_intersections(out) 
        
        return dict_over_ind
    
def calculate_inclusion_indices(g1_reind, g2_reind):

        out = sjoin(g1_reind, g2_reind, op = "contains") ## PP contains SAL
        
        out.drop('index_right', axis=1, inplace=True) 
        
        dict_over_ind = find_intersections(out) 
        
        return dict_over_ind
    
def calculate_join(dict_over_ind, g1_reind, g2_reind):
        area_total = 0
        data_aggreg = []

        # note to self: make sure to import shapely Polygon
        for index1, crim in g1_reind.iterrows():
            try:
                index1 = crim.pp_index
                sals_found = dict_over_ind[index1]

                for sal in range(len(sals_found)):
                    pom = g2_reind[g2_reind.sal_index == sals_found[sal]]['geometry']        

                    #if pom.intersects(crim['geometry']).values[0]:
                    area_int = pom.intersection(crim['geometry']).area.values[0]
                    if area_int>0:
                        area_total += area_int 
                        area_crim = crim['geometry'].area

                        area_popu = pom.values[0].area

                        popu_count = g2_reind[g2_reind.sal_index == sals_found[sal]]['PPL_CNT'].values[0]
                        murd_count = crim['murd_cnt']
                        pol_province = crim['province']
                        popu_frac = (area_int / area_popu) * popu_count# fraction of the pop area contained inside the crim
                        #print(popu_frac)
                        extra_info_col_names = ['DC_NAME','MN_NAME','MP_NAME','PR_NAME','SP_NAME']
                        
                        extra_info_col_codes = ['MN_CODE','MP_CODE','PR_CODE','SAL_CODE','SP_CODE']

                        extra_names = g2_reind[g2_reind.sal_index == sals_found[sal]][extra_info_col_names]#.filter(regex=("NAME"))
                        extra_codes = g2_reind[g2_reind.sal_index == sals_found[sal]][extra_info_col_codes]#.filter(regex=("NAME"))

                        data_aggreg.append({'geometry': pom.intersection(crim['geometry']).values[0], 'id1': index1,\
                                      'id2': sals_found[sal] ,'area_pp': area_crim,'area_sal': area_popu,\
                                  'area_inter': area_int, 'popu_inter' : popu_frac, 'popu_sal': popu_count,\
                                  'murd_cnt': murd_count,'province': pol_province,
                                  'DC_NAME': extra_names.DC_NAME.values[0],\
                                  'MN_NAME': extra_names.MN_NAME.values[0], 'MP_NAME': extra_names.MP_NAME.values[0],\
                                  'PR_NAME': extra_names.PR_NAME.values[0],'SP_NAME': extra_names.SP_NAME.values[0],\
                                  'MN_CODE': extra_codes.MN_CODE.values[0],'MP_CODE': extra_codes.MP_CODE.values[0],\
                                  'PR_CODE': extra_codes.PR_CODE.values[0],'SAL_CODE': extra_codes.SAL_CODE.values[0],\
                                  'SP_CODE': extra_codes.SP_CODE.values[0]} )
            except:
                pass
            
        df_t = gpd.GeoDataFrame(data_aggreg,columns=['geometry', 'id1','id2','area_pp',\
                                       'area_sal','area_inter', 'popu_inter',\
                                       'popu_sal', 'murd_cnt','province','DC_NAME',\
                                       'MN_NAME','MP_NAME','PR_NAME','SP_NAME',\
                                      'MN_CODE','MP_CODE','PR_CODE','SAL_CODE','SP_CODE'])
        #df_t.to_file(out_name)
        return df_t, area_total, data_aggreg

In [43]:
# this function adds the remaining columns, calculates fractions etc
def compute_final_col(df_temp):
    # add population data per police percinct to the main table
    # id1- PP, id2 - SAL
    temp = df_temp.groupby(by=['id1'])['popu_inter'].sum().reset_index()

    data_with_population = pd.merge(df_temp, temp, on='id1', how='outer')\
            .rename(columns={'popu_inter_y':'popu_frac_per_pp', 'popu_inter_x':'popu_inter'})

    # finally, update the murder rate per SAL : id2 is sal's id    

    data_with_population['murd_est_per_int'] = data_with_population['popu_inter']/data_with_population['popu_frac_per_pp']\
               * data_with_population['murd_cnt']
    data_mur_per_int = data_with_population.groupby(by=['id2'])['murd_est_per_int'].sum().reset_index()

    data_mur_per_sal = data_mur_per_int.rename(columns={'murd_est_per_int':'murd_est_per_sal'})

    data_with_population['ratio_per_int'] = data_with_population['popu_inter']/data_with_population['popu_frac_per_pp']\

    data_complete = pd.merge(data_with_population, data_mur_per_sal, on='id2', how='outer')\
            .rename(columns={'id1':'index_PP', 'id2':'index_SAL'})
    return data_complete

# the geometry can be simplified using Line Simpl. algorithms
# data_with_pop['geo_simplified'] = data_with_pop.geometry.simplify(1000)
# for use one switches to a chosen simplified geometry:
# data_with_pop.set_geometry('geo_simplified', inplace=True)

Main functions to find intersection. Files loaded in are the AEA projected shapefiles.

In [4]:
salSHP_upd = 'shapefiles/updated/sal_population_aea.shp'
polSHP_upd = 'shapefiles/updated/polPrec_murd2015_prov_aea.shp'

geo_pol = gpd.GeoDataFrame.from_file(polSHP_upd)
geo_sal = gpd.GeoDataFrame.from_file(salSHP_upd)

geo_pol_reind = geo_pol.reset_index().rename(columns={'index':'pp_index'})
geo_sal_reind = geo_sal.reset_index().rename(columns={'index':'sal_index'})

dict_int = calculate_join_indices(geo_pol_reind,geo_sal_reind)
print("1")
dict_inc = calculate_inclusion_indices(geo_pol_reind, geo_sal_reind)

1
2
3


test on a subset:

In [ ]:
gt1= geo_pol_reind[geo_pol.province=="Free State"].head(n=2)
gt2 = geo_sal_reind[geo_sal_reind.PR_NAME=="Free State"].reset_index()
d = calculate_join_indices(gt1, gt2)

Running the intersections on pre-computed indices:

In [17]:
from timeit import default_timer as timer

start = timer() 

df_inc, sum_area_inc, data_inc = calculate_join(dict_inc, geo_pol_reind, geo_sal_reind)
end = timer()
print("1st", end - start)  

start = timer() 
df_int, sum_area_int, data_int = calculate_join(dict_int, geo_pol_reind, geo_sal_reind)
end = timer()
print("2nd", end - start)  

1st 1544.5270484819994
2nd 2133.577747626994


In [20]:
df_int.head(n=3)

,geometry,id1,id2,area_pp,area_sal,area_inter,popu_inter,popu_sal,murd_cnt,province,DC_NAME,MN_NAME,MP_NAME,PR_NAME,SP_NAME,MN_CODE,MP_CODE,PR_CODE,SAL_CODE,SP_CODE
0,(POLYGON ((179538.273068833 -3355106.778516249...,0,28532,55396371.27358,1.981387e+08,17779.878339,0.003230,36,25,Free State,Mangaung,Mangaung,Mangaung NU,Free State,Mangaung NU,499,499002,4,4990011,499002001
1,POLYGON ((181104.0571247448 -3347250.428848253...,0,29666,55396371.27358,7.408164e+06,7337878.480101,268.428873,271,25,Free State,Mangaung,Mangaung,Botshabelo,Free State,Botshabelo SP,499,499030,4,4990063,499030008
2,POLYGON ((176437.6520079001 -3347422.027408466...,0,29636,55396371.27358,2.894491e+05,289449.084550,707.000000,707,25,Free State,Mangaung,Mangaung,Botshabelo,Free State,Botshabelo G,499,499030,4,4991112,499030005


find pol precincts within WC boundary

In [ ]:
za_province = gpd.read_file('za-provinces.topojson',driver='GeoJSON')#.set_index('id')
za_province.crs={'init': '27700'}

wc_boundary = za_province.ix[8].geometry # WC
#pp_WC = geo_pol[geo_pol.geometry.within(wc_boundary)]
pp_WC_in = geo_pol[geo_pol.geometry.intersects(wc_boundary)]
#.unary_union, sal_wc_union_bound = sal_WC_in.unary_union
pp_WC_overlaps = pp_WC_in[pp_WC_in.province!="Western Cape"]
pp_WC_pol_annot = pp_WC_in[pp_WC_in.province=="Western Cape"]

In [ ]:
#pp_test = pp_WC_in[pp_WC_in['compnt_nm'].isin(['atlantis','philadelphia','kraaifontein','brackenfell','kuilsriver','kleinvleveerste river','macassar','somerset west','fish hoek'])]
#pp_test = pp_WC_in[pp_WC_in['compnt_nm'].isin(['beaufort west','doring bay','murraysburg', 'strandfontein','nuwerus','lutzville'])]
%matplotlib inline
pp_WC_overlaps.plot()

To compare, we load the data without the AEA projection:

In [ ]:
df_int_obs =gpd.GeoDataFrame.from_file('data/intersection_int_ext.shp')

In [ ]:
dfFS = df[df.PR_NAME=="Free State"]
dfoFS = df_int_obs[df_int2.PR_NAME=="Free State"]

In [ ]:
dfoFS[dfoFS.id1 == 0]['popu_frac']

In [ ]:
dfFS[dfFS.id1 == 0]['popu_frac'].sum()

Add the final columns and save the files +  check the sums across provinces.
First select only the SALs that are included at least 50% in the intersection

In [78]:
df_int.head(n=2)

,geometry,id1,id2,area_pp,area_sal,area_inter,popu_inter,popu_sal,murd_cnt,province,DC_NAME,MN_NAME,MP_NAME,PR_NAME,SP_NAME,MN_CODE,MP_CODE,PR_CODE,SAL_CODE,SP_CODE
0,(POLYGON ((179538.273068833 -3355106.778516249...,0,28532,55396371.27358,1.981387e+08,17779.878339,0.003230,36,25,Free State,Mangaung,Mangaung,Mangaung NU,Free State,Mangaung NU,499,499002,4,4990011,499002001
1,POLYGON ((181104.0571247448 -3347250.428848253...,0,29666,55396371.27358,7.408164e+06,7337878.480101,268.428873,271,25,Free State,Mangaung,Mangaung,Botshabelo,Free State,Botshabelo SP,499,499030,4,4990063,499030008


Additing additional columns and computing the lower/upper estimation bounds (selecting SALs overlapping at >50% or completely):

In [84]:
#list_mit =[]
list_int_inc = []
for i,entry in df_int.iterrows():#f_inc_aea:
    #if (entry.area_inter/entry.area_sal>=0.5): # select intersections where SAL overlap >50%
     #   list_mit.append(entry)
    if (entry.area_inter/entry.area_sal==1): # select those included 'completely'
        list_int_inc.append(entry)
df_int_mit=gpd.GeoDataFrame(list_mit)
df_int_inc=gpd.GeoDataFrame(list_int_inc)

In [82]:
df_int_aea_mit = compute_final_col(df_int_mit) # mitad= mid
df_int_aea_cpt = compute_final_col(df_int_inc) # complete

In [87]:
df_int_aea_mit.to_csv('data/pp_sal_aea_mit.csv')
df_int_aea_cpt.to_csv('data/pp_sal_aea_cpt.csv')

In [25]:
df_int_aea = compute_final_col(df_int) # all intersections
#df_inc_aea = compute_final_col(df_inc) # all full-inclusions
#df_inc_aea.to_csv('data/pp_sal_inc_included.csv')
df_int_aea.to_csv('data/pp_sal_inc_intersections.csv')

In [33]:
data_prov = df_int_aea[['PR_NAME','province','murd_est_per_int']]
data_prov.groupby('province')['murd_est_per_int'].sum()

province
0                   0
Eastern Cape     3051
Free State        943
Gauteng          3671
Kwazulu/Natal    3759
Limpopo           777
Mpumalanga        831
North West        853
Northern Cape     411
Western Cape     3186
Name: murd_est_per_int, dtype: float64

In [29]:
data_prov.groupby('PR_NAME')['murd_est_per_int'].sum()

PR_NAME
Eastern Cape     3051.126640
Free State        942.827525
Gauteng          3654.691802
KwaZulu-Natal    3758.865333
Limpopo           775.293714
Mpumalanga        832.509613
North West        872.349383
Northern Cape     408.343123
Western Cape     3185.992867
Name: murd_est_per_int, dtype: float64

In [31]:
# check over small areas- sum of all the crimes should be 17482
pom = {}
for ind, row in df_inc_aea.iterrows():
    pom[row['index_SAL']] = row['murd_est_per_sal'] 
s=0
for key in pom:
    s = s + pom[key]
print(s)

17482.00000000003


Computing the bounds:

In [47]:
len(df_inc_aea)

71594

In [39]:
len(df_int)

101543

In [48]:
data_prov_ub = df_inc_aea[['PR_NAME','province','murd_est_per_int']]
data_prov_ub.groupby('province')['murd_est_per_int'].sum()

province
0                   0
Eastern Cape     3047
Free State        940
Gauteng          3670
Kwazulu/Natal    3758
Limpopo           777
Mpumalanga        830
North West        846
Northern Cape     410
Western Cape     3186
Name: murd_est_per_int, dtype: float64

In [35]:
data_prov = df_int_aea[['PR_NAME','province','murd_est_per_int']]
data_prov.groupby('province')['murd_est_per_int'].sum()

province
0                   0
Eastern Cape     3051
Free State        943
Gauteng          3671
Kwazulu/Natal    3759
Limpopo           777
Mpumalanga        831
North West        853
Northern Cape     411
Western Cape     3186
Name: murd_est_per_int, dtype: float64

In [ ]:
#f, ax = plt.subplots(1, figsize=(12, 12))
#wc.plot(ax=ax, color='grey', linewidth=0)
#gpd.plotting.plot_multipolygon(ax, sal_wc_union_bound, facecolor='green')

#gpd.plotting.plot_multipolygon(ax, sal_wc_union, facecolor='blue')

In [ ]:
f, ax = plt.subplots(1, figsize=(12, 12))
gpd.plotting.plot_multipolygon(ax, pp_WC_overlaps.unary_union, facecolor='red')
wc.plot(ax=ax, color='black', linewidth=0) 

In [49]:
df_inc_aea.head()

,geometry,index_PP,index_SAL,area_pp,area_sal,area_inter,popu_inter,popu_sal,murd_cnt,province,...,SP_NAME,MN_CODE,MP_CODE,PR_CODE,SAL_CODE,SP_CODE,popu_frac_per_pp,murd_est_per_int,ratio_per_int,murd_est_per_sal
0,POLYGON ((176437.6520079001 -3347422.027408466...,0,29636,55396371.27358,289449.084550,289449.084550,707,707,25,Free State,...,Botshabelo G,499,499030,4,4991112,499030005,68494,0.258052,0.010322,0.258052
1,POLYGON ((176244.7363563863 -3347919.723384398...,0,29632,55396371.27358,204893.780442,204893.780442,618,618,25,Free State,...,Botshabelo G,499,499030,4,4990875,499030005,68494,0.225567,0.009023,0.225567
2,POLYGON ((176363.9246824788 -3350001.103421158...,0,29644,55396371.27358,147431.344571,147431.344571,535,535,25,Free State,...,Botshabelo C,499,499030,4,4990568,499030007,68494,0.195273,0.007811,0.195273
3,POLYGON ((176601.5611199073 -3350155.867966976...,0,29656,55396371.27358,131466.444886,131466.444886,721,721,25,Free State,...,Botshabelo C,499,499030,4,4991128,499030007,68494,0.263162,0.010526,0.263162
4,POLYGON ((176846.4617087045 -3350388.902318581...,0,29642,55396371.27358,115324.558937,115324.558937,473,473,25,Free State,...,Botshabelo C,499,499030,4,4990365,499030007,68494,0.172643,0.006906,0.172643


In [56]:
df_inc_aea[df_inc_aea.index_SAL==29656].murd_est_per_sal

3    0.263162
Name: murd_est_per_sal, dtype: float64

In [57]:
df_int_aea[df_int_aea.index_SAL==29656].murd_est_per_sal

15    0.22013
Name: murd_est_per_sal, dtype: float64

In [58]:
len(set(df_int_aea.index_SAL))

84907

In [60]:
df_inc_aea.head(n=2)

,geometry,index_PP,index_SAL,area_pp,area_sal,area_inter,popu_inter,popu_sal,murd_cnt,province,...,SP_NAME,MN_CODE,MP_CODE,PR_CODE,SAL_CODE,SP_CODE,popu_frac_per_pp,murd_est_per_int,ratio_per_int,murd_est_per_sal
0,POLYGON ((176437.6520079001 -3347422.027408466...,0,29636,55396371.27358,289449.084550,289449.084550,707,707,25,Free State,...,Botshabelo G,499,499030,4,4991112,499030005,68494,0.258052,0.010322,0.258052
1,POLYGON ((176244.7363563863 -3347919.723384398...,0,29632,55396371.27358,204893.780442,204893.780442,618,618,25,Free State,...,Botshabelo G,499,499030,4,4990875,499030005,68494,0.225567,0.009023,0.225567


In [61]:
d = df_inc_aea.head(n=20)

WARDS:

In [ ]:
wardsShp =gpd.GeoDataFrame.from_file('../wards_delimitation/Wards_demarc/Wards2011.shp')

In [ ]:
wardsShp.head()

In [ ]:
wardsShp.crs